In [3]:
#最高版本 
"""
未设置checkpoint
"""

"""
1.改进：f
2.category中unique较多元素设置为0,然后label的时候方便判断
3.检查feature前的特征值，做清洗。
"""

"""
此模型利用了xgboost进行分类，详细调优过程请参看代码和注释。
"""

#读取数据集
import pandas as pd
train= pd.read_csv('./train.csv',encoding='utf-8')
test = pd.read_csv('./test1.csv',encoding='utf-8')
features = train.drop(['Unnamed: 0','label'],axis=1)
labels = train['label']


In [4]:

#fea1_hash和fea_hash有不同长度，可能代表着用户行为的多少，此处用fea1_hash的长度构建新特征，模型准确度有一定提高（后期可根据不同的数字序列尝试再分类）
features['fea1_hash4']=features['fea1_hash'].map(lambda x:len(str(x)))
train['fea1_hash4']=test['fea1_hash'].map(lambda x:len(str(x)))    
test['fea1_hash4']=test['fea1_hash'].map(lambda x:len(str(x)))

#新特征函数，利用数量特征构造。
def de(x):
    result = pd.value_counts(x)
    x= [999999999 if result[each]==1  else each for each in x]
    return x

def ded(x):
    result = pd.value_counts(x)
    x= [result[each] for each in x]
    return x

#合并测试集和训练集
all_df = pd.concat([train,test])

#利用数量特征构造新特征--采用唯一值较多的特征，因为有重复id或重复行为点击广告表明很可能不是真人。
all_df.android_id=de(all_df.android_id)
all_df.fea_hash=de(all_df.fea_hash)
all_df.fea1_hash=de(all_df.fea1_hash)

features.android_id = all_df[all_df['label'].notnull()]['android_id']
features.fea_hash = all_df[all_df['label'].notnull()]['fea_hash']
features.fea1_hash = all_df[all_df['label'].notnull()]['fea1_hash']

test.android_id = all_df[all_df['label'].isnull()]['android_id']
test.fea_hash = all_df[all_df['label'].isnull()]['fea_hash']
test.fea1_hash = all_df[all_df['label'].isnull()]['fea1_hash']

train.android_id = all_df[all_df['label'].notnull()]['android_id']
train.fea_hash = all_df[all_df['label'].notnull()]['fea_hash']
train.fea1_hash = all_df[all_df['label'].notnull()]['fea1_hash']

#尝试出构造数量特征后有提升的原有特征
s = ['dev_height','dev_width','media_id','package','apptype','android_id','fea1_hash']

for f in s:
    all_df[f]=ded(all_df[f])
    train[f+'2'] = all_df[all_df['label'].notnull()][f]
    test[f+'2'] = all_df[all_df['label'].isnull()][f]
    features[f+'2'] = all_df[all_df['label'].notnull()][f]


In [5]:
#数据清洗osv
def f(x):
    if str(x) == 'nan':
        return x
    else:x = str(x)
    y = x
    r = '';
    for i in range(len(x)):
        if x[i].isdigit():
            r +=x[i]
    if r == '':
        return 0
    else:
        while(r[0]=='0'):
            r=r[1:]
    k = int((str(r)+'000')[:5])
    while(k>12):
        k=k/10
    return float(k)
    
train['osv'] = train['osv'].apply(f)
features['osv'] = features['osv'].apply(f)
test['osv'] = test['osv'].apply(f)


#类别特征，后期统一labelencoder
cate_features = ['apptype','carrier','ntt','location','cus_type','media_id',
'dev_width','dev_height','android_id','fea1_hash']

pd.set_option('display.max_rows',100)

In [6]:
from datetime import datetime as dt 
"""
个人处理日期的想法：按照kmeans，但是是一维，可以直接sort聚类
"""
#处理日期，得到基于小时数的timediff
def get_date(features):
    features['timestamp'] = features['timestamp'].apply(lambda x: dt.fromtimestamp(x/1000))
    start_time = features['timestamp'].min()
    features['time_diff'] = features['timestamp'] - start_time
    features['time_diff'] = features['time_diff'].dt.days*24 + features['time_diff'].dt.seconds/3600
    features.drop(['timestamp'],axis=1,inplace = True)
    return features

features = get_date(features)
test = get_date(test)
train = get_date(train)

mini = features['time_diff'].min()

#尝试出不同的聚类距离，13时提升效果最大
def ts(x):
    return (x - mini)//13

features['time_diff'] = features['time_diff'].apply(ts)
train['time_diff'] = train['time_diff'].apply(ts)
test['time_diff'] = test['time_diff'].apply(ts)

cate_features.append('time_diff')

In [7]:
#去掉过长的特征（极大值）
features['fea_hash']=features['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
features['fea1_hash']=features['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
test['fea_hash']=test['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
test['fea1_hash']=test['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
train['fea_hash']=test['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
train['fea1_hash']=test['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#特征清洗lan
def low(row):
    return row.lower().replace('-',"").replace('_',"")

#将训练集和测试集合并
all_df = pd.concat([train,test])
all_df['lan']= all_df['lan'].astype('str').apply(low)
all_df['lan'] = le.fit_transform(all_df['lan'])
all_df['fea_hash']= all_df['fea_hash'].astype('str')
all_df['fea_hash'] = le.fit_transform(all_df['fea_hash'])



In [8]:
#移除不必要的特征
nonuse = ['os','sid']
col = features.columns.tolist()
for i in nonuse:
    col.remove(i)
features = features[col]

In [9]:
#特征清洗version
def rep(x):
    if str(x).isdigit():return int(x)
    elif str(x)[0] == "v" or "V":
        if str(x)[1:].isdigit():
            return int(str(x)[1:])
    else:return 0
    
features['version'] = features['version'].apply(rep)
all_df['version'] = all_df['version'].apply(rep)
train['version'] = train['version'].apply(rep)

#统一labelencoder
for fea in cate_features:
    all_df[fea]= all_df[fea].astype('float')
    all_df[fea] = le.fit_transform(all_df[fea])
    
features['lan'] = all_df[all_df['label'].notnull()]['lan']
features['fea_hash'] = all_df[all_df['label'].notnull()]['fea_hash']

train['lan'] = all_df[all_df['label'].notnull()]['lan']
train['fea_hash'] = all_df[all_df['label'].notnull()]['fea_hash']

for fea in cate_features:
    features[fea] = all_df[all_df['label'].notnull()][fea]
    train[fea] = all_df[all_df['label'].notnull()][fea]

test_fea = test[features.columns]

test_fea['lan'] = all_df[all_df['label'].isnull()]['lan']
test_fea['fea_hash'] = all_df[all_df['label'].isnull()]['fea_hash']
test_fea['version'] = test_fea['version'].apply(rep)

for fea in cate_features:
    test_fea[fea] = all_df[all_df['label'].isnull()][fea]

cate_features.append('lan')
features['version']=features['version'].astype(float)



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [10]:
#测试出dev_ppi会使效果下降，放弃使用这个特征
f1 = features.drop(['dev_ppi'],axis=1)

#挑选出影响最大的特征
selected_c = f1.columns
def find_key_f(train,selected):
    temp0 = train[train['label']==0]
    temp = pd.DataFrame(columns=[0,1])
    temp[0] = temp0[selected].value_counts()/len(temp0) *100
    temp1 = train[train['label']==1]
    temp[1] = temp1[selected].value_counts()/len(temp0) *100
    temp[2] = temp[1]/temp[0]
    result = temp[temp[2]>7].sort_values(2,ascending = False).index
    return result

kf = {}
for selected in selected_c:
    kf[selected] = find_key_f(train,selected)

#挑选出影响最大的特征值
def ff(x,selected):
    if x in kf[selected]:
        return 1
    else:
        return 0

for selected in selected_c:
    if len(kf[selected])>0:
        features[selected+'1'] = features[selected].apply(ff,args = (selected,))
        test_fea[selected+'1'] = test_fea[selected].apply(ff,args = (selected,))
        print(selected)



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


android_id
apptype
dev_height
dev_width
media_id
ntt
osv
package
dev_height2
dev_width2
media_id2
package2
apptype2
android_id2
fea1_hash2


In [11]:
#五折交叉验证
from sklearn.model_selection import  StratifiedKFold,KFold
from sklearn.metrics import accuracy_score
def ensemble(clf, train_x, train_y, test, cate_features):
    prob = [] 
    mean_acc = 0 
    sk = StratifiedKFold(n_splits=5,shuffle=True,random_state=2021)
    for k, (train_i, val_i) in enumerate(sk.split(train_x,train_y)):
        train_x_real = train_x.iloc[train_i]
        train_y_real = train_y.iloc[train_i]
        val_x = train_x.iloc[val_i]
        val_y = train_y.iloc[val_i]
        clf = clf.fit(train_x_real,train_y_real)
        val_y_pred = clf.predict(val_x)
        acc_val = accuracy_score(val_y,val_y_pred)
        print("第{}个子模型 acc={}".format(k+1,acc_val))
        mean_acc += acc_val/5
        test_y_pred = clf.predict_proba(test)[:-1]
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob = sum(prob) / 5
    return mean_prob

In [12]:
#XGB调参
import xgboost as xgb
clf = xgb.XGBClassifier(
            max_depth=13, learning_rate=0.005, n_estimators=2300, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.95, colsample_bytree=0.4, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5,
        )
ensemble(clf, features, labels, test_fea, cate_features)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:08:53] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第1个子模型 acc=0.89272
[17:13:14] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第2个子模型 acc=0.8921
[17:17:36] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. 

array([[0.93963397, 0.06036603],
       [0.28671676, 0.71328324],
       [0.9747782 , 0.02522179],
       ...,
       [0.04511253, 0.9548875 ],
       [0.07228599, 0.927714  ],
       [0.03320142, 0.9667986 ]], dtype=float32)

In [13]:
#预测结果
a = pd.DataFrame(test['sid'])
a['label']= clf.predict(test_fea)
a.to_csv('89.252.csv',index = False)